In [419]:
from math import sqrt
import numpy as np
import numpy.linalg as nla
import numpy.polynomial.laguerre as nplag

In [369]:
def householder(x,i):
    
    n = x.shape[0]
    dot_1on = np.dot(x,x) - x[i]**2

    v = np.copy(x)
    v[i] = 1.0
    
    if dot_1on < np.finfo(float).eps:
        beta = 0.0
    else:
        norm_x = sqrt(x[i]**2 + dot_1on)
        if x[i] <= 0:
            v[i] = x[i] - norm_x
        else:
            v[i] = -dot_1on / (x[i] + norm_x)
        beta = 2 * v[i]**2 / (dot_1on + v[i]**2)
        v = v / v[i]
    
    return np.eye(n) - beta * np.outer(v,v)


In [406]:
# polyharmonic spline rbf
def phs(d,k):
    
    if k%2 == 0:
        if d == 0:
            return 0
        else:
            return (d**k) * np.log(d)
    else:
        return d**k

# 1st order differential operator of polyharmonic spline rbf
def diff_phs(d,d_i,k):
    
    if k%2 == 0:
        if d == 0:
            df = 0
        else:
            df = d_i * (d**(k-2)) * (1 + (k * np.log(d)))
    else:
        df = k * d_i * (d**(k-2))

    return df

def GA_rbf(r,ep):
    
    return np.exp(-(ep*r)**2)

def GA_rbf_lap(r,ep,k):
    
    return ep**(2*k) * nplag.lagval(r,np.eye(1,k,k-1).reshape((-1))) * GA_rbf(r,ep)


In [420]:
def get_rbffd_weights(X,D,N):
    
    k = 3
    n = X.shape[1]
    
    # apply householder transformation to center stencil on x-axis
    H = householder(X[:,0],2)
    Xp = np.matmul(H,X)
    
    # Create A Matrix
    A_rbf = phs(D,k)
    
    # Polynomial part
    
    N_poly = 21
    
    x = Xp[0].reshape((1,-1))
    y = Xp[1].reshape((1,-1))
    A_poly = np.concatenate((np.ones((1,n)),x,y,x**2,x*y,y**2,x**3,(x**2)*y,x*(y**2),y**3,x**4,(x**3)*y,(x**2)*(y**2),x*(y**3),y**4,x**5,(x**4)*y,(x**3)*(y**2),(x**2)*(y**3),x*(y**4),y**5),axis=0)

    # create full PHS-RBF Matrix
    A = np.concatenate((np.concatenate((A_rbf,np.transpose(A_poly)),axis=1),np.concatenate((A_poly,np.zeros((N_poly,N_poly))),axis=1)),axis=0)

    # for d/dx
    b_x = np.concatenate((diff_phs(D[0],x[0,0]-x[0],k),np.eye(1,N_poly,1).reshape((-1))),axis=0)
    w_x = nla.solve(A,b_x)[0:n]
    
    # for d/dy
    b_y = np.concatenate((diff_phs(D[0],y[0,0]-y[0],k),np.eye(1,N_poly,2).reshape((-1))),axis=0)
    w_y = nla.solve(A,b_y)[0:n]
    
    # reverse householder transformation to get final weights
    Wp = np.concatenate((w_x,w_y,np.zeros(n)),axis=0).reshape((3,-1))
    W = (np.matmul(H,Wp))
         
    FDV = np.append(np.zeros((2,7)),np.array([[-1/60,3/20,-3/4,0,3/4,-3/20,1/60]]),axis=0)
    FDV = np.matmul(H,FDV)
    
    return W,FDV

In [411]:
def get_RBFFD_DMs(X,n):
    # xx: 2D FP array (3xNnodes) describing the nodeset in cartesian coordinates
    # n: stencil size
    
    # get number of nodes
    N = X.shape[1]
    
    # tensor product to "meshgrid" coordinates
    xx,yy,zz = np.tensordot(X, np.ones(N), 0)
    
    # distance matrix
    D = np.sqrt((xx - xx.T)**2 + (yy - yy.T)**2 + (zz - zz.T)**2)
    
    # get n-point stencils
    idx = np.argsort(D)[:,0:n]
    
    # for each stencil get associated RBFFD differentiation weights
    for i in range(N):
        
        DM_i,FDV_i = get_rbffd_weights(X[:,idx[i]],(D[idx[i]])[:,idx[i]],N)
        DM_i = DM_i.reshape((1,3,n))
        FDV_i = FDV_i.reshape((1,3,7))
        
        if i == 0:
            DMs = DM_i
            FDVs = FDV_i
        else:
            DMs = np.concatenate((DMs,DM_i),axis=0)
            FDVs = np.concatenate((FDVs,FDV_i),axis=0)
            
        if i%int(N/10) == 0:
            print("get_RBFFD_DMs status: node_id =",i)
            
    return DMs,FDVs,idx


In [ ]:
def get_full_DMs(DM,FDV,idx,Nv,depth,h):
    
    Nh = DM.shape[0]
    n = DM.shape[2]
    
    Ntot = (Nv + (2*depth)) * Nh
    
    Dx = np.zeros((Ntot,Ntot))
    Dy = np.zeros((Ntot,Ntot))
    Dz = np.zeros((Ntot,Ntot))
    
    for i in range(depth,Nv+depth):

        for j in range(Nh):
            for k in range(n):
                
                Dx[(i*Nh)+j,(i*Nh)+idx[j,k]] = DM[j,0,k]/rlvls[i]
                Dy[(i*Nh)+j,(i*Nh)+idx[j,k]] = DM[j,1,k]/rlvls[i]
                Dz[(i*Nh)+j,(i*Nh)+idx[j,k]] = DM[j,2,k]/rlvls[i]
                
            for k in range(-depth,depth):
                Dx[(i*Nh)+j,((i+k)*Nh)+j] += FDV[j,0,k+depth]/h
                Dy[(i*Nh)+j,((i+k)*Nh)+j] += FDV[j,1,k+depth]/h
                Dz[(i*Nh)+j,((i+k)*Nh)+j] += FDV[j,2,k+depth]/h
    
    for i in range(depth):
        
        i1 = i
        i2 = (2*depth) - i
        Dx[i1*Nh:(i1+1)*Nh,:] = Dx[i2*Nh:(i2+1)*Nh,:]
        Dy[i1*Nh:(i1+1)*Nh,:] = Dy[i2*Nh:(i2+1)*Nh,:]
        Dz[i1*Nh:(i1+1)*Nh,:] = Dz[i2*Nh:(i2+1)*Nh,:]
        
        i2 = (Nv - 1) + i
        i1 = (Nv + (2*depth) - 1) - i
        Dx[i1*Nh:(i1+1)*Nh,:] = Dx[i2*Nh:(i2+1)*Nh,:]
        Dy[i1*Nh:(i1+1)*Nh,:] = Dy[i2*Nh:(i2+1)*Nh,:]
        Dz[i1*Nh:(i1+1)*Nh,:] = Dz[i2*Nh:(i2+1)*Nh,:]
    
    return Dx,Dy,Dz